In [1]:
# 앞절에서 수행한 내용들

import os
import numpy as np
from sklearn.datasets import fetch_openml

dataset_root = os.path.join(os.getcwd(), 'datasets')
if not os.path.isdir(dataset_root):
    os.mkdir(dataset_root)

    
mnist = fetch_openml('mnist_784', version=1, data_home=dataset_root)
X, y = mnist["data"], mnist["target"].astype(np.uint8)

X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

# Multiclass Classification

**Multiclass classifier(multinomial classifier)**은 2개보다 많은 class를 분류한다.


Multiclass를 바로 다룰 수 있는 알고리즘(random forest classifier 또는 naive bayes classifier 등)도 있지만, 여러개의 binary classfier를 사용해 multiclass classification을 수행하는 알고리즘(support vector machine classifer 또는 linear classifier 등)도 있다.

binary classifier를 사용한 방법중 하나는 예를 들어, 10개의 숫자를 분류한다면, 10개의 classifier를 사용해 해당 숫자인지 아닌지를 분류하는 것이다.(0-detector 1-detector, $\cdots$, 9-detector) 이 방법은 **one-versus-all(OvA)** 또는 **one-versus-the-rest**라고 한다.

또 다른 방법은 모든 숫자 쌍에 대한 classifier를 학습시키는 것이다.(0과1, 0과2, $\cdots$, 8과 9) 이 방법은 **one-versus-one(OvO)**라고 한다. N개의 class를 분류한다면, N*(N-1)/2개의 분류기를 학습시키게 된다.

대부분의 binary classification 알고리즘은 OvA가 선호되지만, 몇몇 알고리즘(support vector machine classifier 등)은 training set의 크기에 따라 scale이 매우 커지므로, 이러한 알고리즘은 큰 training set으로 적은 classifer를 학습시키는 것보다, 작은 training set으로 여러 classifier를 학습시키는 것이 더욱 빠르므로, OvO를 사용하는 것이 더 낫다.

사이킷런은 binary classification 알고리즘으로 multiclass classification을 수행하려고 하면, 이를 감지해서 자동으로 OvA 방법을 사용한다.(SVM은 OvO를 사용)

다음은 multiclass classification에 `SGDClassifier`를 사용하는 예제이다.

In [2]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train)

some_digit = X_train[10]
some_digit_answer = y_train[10]
print(f"prediction: {sgd_clf.predict([some_digit])}, answer: {[some_digit_answer]}")

prediction: [3], answer: [3]


위 코드를 수행하면 사이킷런은 10개의 binary classifier를 학습시킨다. 그리고 prediction시, 10개의 decision score에서 높은 score의 class를 선택한다.

각 score와 가장 높은 점수의 class는 `decision_score()`와 `argmax()`를 통해 다음과 같이 확인할 수 있다.

In [3]:
some_digit_scores = sgd_clf.decision_function([some_digit])
print(some_digit_scores)

[[-32927.58071595 -17846.08012197  -9117.15524931   8110.41175195
  -20824.50629491  -7276.9608832  -23298.24023431 -21227.32271866
   -3488.10980768  -5439.38177054]]


In [4]:
print(np.argmax(some_digit_scores))

3


또한, classifier의 class는 다음과 같이 확인할 수 있다.

In [5]:
print(sgd_clf.classes_)

[0 1 2 3 4 5 6 7 8 9]


OvO나 OvA를 직접 지정하고 싶다면, `OneVsOneClassifier`와 `OneVsRestClassifier` class를 사용하면 된다.

다음은 SGDClassifier를 OvO로 학습시킨 예이다.

In [6]:
from sklearn.multiclass import OneVsOneClassifier
ovo_clf = OneVsOneClassifier(SGDClassifier(random_state=42))
ovo_clf.fit(X_train, y_train)
print(f"prediction: {ovo_clf.predict([some_digit])}, answer: {[some_digit_answer]}")

prediction: [3], answer: [3]


classifier의 수는 다음과 같이 확인할 수 있다.

In [7]:
print(f"num_estimator: {len(ovo_clf.estimators_)}")

num_estimator: 45


Multiclass를 바로 다룰 수 있는 `RandomForestClassifier`를 학습시키는 것은 간단하다.

In [8]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state=42)
forest_clf.fit(X_train, y_train)
print(f"prediction: {forest_clf.predict([some_digit])}, answer: {[some_digit_answer]}")

prediction: [3], answer: [3]


`predict_proba()`를 통해 classifier가 할당한 각 class별 확률을 출력할 수 있다.

In [9]:
print(forest_clf.predict_proba([some_digit]))

[[0.   0.   0.   0.99 0.   0.01 0.   0.   0.   0.  ]]


이제 classifier의 성능을 평가해보자. 평소와 같이 cross_validation을 사용하면 된다.

다음은 `SGDClassifier`의 정확도를 `cross_val_score()`로 평가하는 예이다.

In [10]:
from sklearn.model_selection import cross_val_score

print(cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring="accuracy"))

[0.87365 0.85835 0.8689 ]


나쁘지 않은 성능이지만, 더 높이고 싶다면, 2장에서처럼 scaling을 해주는 방법이 있다.

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
print(cross_val_score(sgd_clf, X_train_scaled, y_train, cv=3, scoring="accuracy"))

[0.8983 0.891  0.9018]
